###Pruebas y optimizaciones del cálculo de K y K' del código de kzapfre https://github.com/kzapfe/RGutWork/blob/master/kCSD/ObtenciosdeLaMatrizKyKtilde.ipynb

In [3]:
cd("/Users/Nes/Desktop/Neuroscience/Parallel-Neuroscience/P2kCSDA");

In [4]:
pwd()

"/Users/Nes/Desktop/Neuroscience/Parallel-Neuroscience/P2kCSDA"

In [6]:
ConjuntoDeCoordenadasTotal=Array[]

0-element Array{Array{T,N},1}

In [7]:
for j=1:4096
    push!(ConjuntoDeCoordenadasTotal,[convert(Int64,ceil(j /64)) ,convert(Int64,j % 64)])
end
ConjuntoDeCoordenadasTotal[65]

2-element Array{Int64,1}:
 2
 1

In [1]:
BceroDura=readdlm("BceroDura.dat");

###Sobre escribimos el valor -Inf de la integral en el origen (error)

In [8]:
BceroDura[64,64]=137.089

137.089

In [9]:
writedlm("BceroDura.dat", BceroDura)

In [10]:
BceroDura=readdlm("BceroDura.dat");

In [11]:
BceroDura[64,64]

137.089

###Prueba cuando las coordenadas se toman de un conjunto para 128 electrodos

In [8]:
jmax=128
xconstante=[64,64]
Kduras=zeros(jmax,jmax);
@time for j=1:jmax
     xj=ConjuntoDeCoordenadasTotal[j]
     for k=1:j
            xk=ConjuntoDeCoordenadasTotal[k]
        for l=1:jmax
            xl=ConjuntoDeCoordenadasTotal[l]
            Kduras[j,k]+=BceroDura[(xk-xl+xconstante)...]*BceroDura[(xj-xl+xconstante)...]
            end
    end
end

  3.308167 seconds (25.38 M allocations: 919.637 MB, 2.75% gc time)


###Reemplazamos por cálculo de coordenadas con módulo y división

In [17]:
[convert(Int64,ceil(3274 /64)) ,convert(Int64,3274 % 64)]

2-element Array{Int64,1}:
 52
 10

In [4]:
jmax=128
xconstante=[64,64]
KdurasCalc=zeros(jmax,jmax);

@time for j=1:jmax
     xj=[convert(Int64,ceil(j /64)) ,convert(Int64,j % 64)]
     for k=1:j
            xk=[convert(Int64,ceil(k /64)) ,convert(Int64,k % 64)]
        for l=1:jmax
            xl=[convert(Int64,ceil(l /64)) ,convert(Int64,l % 64)]
            KdurasCalc[j,k]+=BceroDura[(xk-xl+xconstante)...]*BceroDura[(xj-xl+xconstante)...]
            end
    end
end


  3.665136 seconds (28.57 M allocations: 1.009 GB, 2.40% gc time)


In [24]:
ConjuntoDeCoordenadasTotal=Array[]
for j=1:4096
    push!(ConjuntoDeCoordenadasTotal2,[convert(Int64,ceil(j /64)) ,convert(Int64,j % 64)])
end

In [9]:
Kduras-KdurasCalc

128x128 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.

In [10]:
length(BceroDura)

16129

In [13]:
size(BceroDura)

(127,127)

In [14]:
size(Kduras)

(128,128)

In [16]:
jmax=4096
KdurasParalel=SharedArray(Float64, jmax,jmax);

In [17]:
size(KdurasParalel)

(4096,4096)